In [1]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from dataset import KeypointRandDataset, KeypointRotDataset
import albumentations as A


In [2]:
import importlib
import dataset

# Reload the module
importlib.reload(dataset)

train_json = "train/_mediapipe_annotated.json"
val_json = "valid/_mediapipe_annotated.json"
test_json = "test/_mediapipe_annotated.json"
input_dim = 63  # 21 keypoints * 3 coordinates
hidden_dim = 256


In [3]:
num_classes = 26  # Adjust based on your dataset

# Dataset and DataLoader
train_dataset = KeypointRotDataset(train_json,  True)
val_dataset = KeypointRotDataset(val_json, True)
test_dataset = KeypointRotDataset(test_json, True)

In [4]:
train_keypoints = train_dataset.keypoints
X_train = train_keypoints - train_keypoints[:, [9]]
# X_train = X_train[:, :, :2].reshape(-1, 42)
X_train = X_train[:, :, :3].reshape(-1, 63)
y_train = train_dataset.categories

val_keypoints = val_dataset.keypoints
X_val = val_keypoints - val_keypoints[:, [9]]
# X_val = X_val[:, :, :2].reshape(-1, 42)
X_val = X_val[:, :, :3].reshape(-1, 63)
y_val = val_dataset.categories

test_keypoints = test_dataset.keypoints
X_test = test_keypoints - test_keypoints[:, [9]]
# X_test = X_test[:, :, :2].reshape(-1, 42)
X_test = X_test[:, :, :3].reshape(-1, 63)
y_test = test_dataset.categories

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape)

(1205, 63) (1205,) (61, 63) (61,) (133, 63) (133,)


In [5]:
# # plot an example
# eg = X_train[0]
# plt.scatter(eg[:, 0], eg[:, 1])
# plt.xlim(-1, 1)
# plt.ylim(-1, 1)

In [6]:
# train a randomforest
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

rf = RandomForestClassifier(random_state=42, n_estimators=200, n_jobs=-1)
rf.fit(X_train, y_train)

print(accuracy_score(rf.predict(X_val), y_val))
print(accuracy_score(rf.predict(X_test), y_test))
print(accuracy_score(rf.predict(X_train), y_train))

# print(classification_report(rf.predict(X_val), y_val))
# print(classification_report(rf.predict(X_test), y_test))
# print(classification_report(rf.predict(X_train), y_train))

# ex = ExtraTreesClassifier(random_state=42, n_estimators=100, n_jobs=-1)
# ex.fit(X_train, y_train)


# print(classification_report(ex.predict(X_val), y_val))
# print(classification_report(ex.predict(X_test), y_test))
# print(classification_report(ex.predict(X_train), y_train))

0.8270676691729323
0.819672131147541
1.0


In [7]:
svc = SVC(random_state=42)
svc.fit(X_train, y_train)

print(accuracy_score(svc.predict(X_val), y_val))
print(accuracy_score(svc.predict(X_test), y_test))
print(accuracy_score(svc.predict(X_train), y_train))

# print(classification_report(svc.predict(X_val), y_val))
# print(classification_report(svc.predict(X_test), y_test))
# print(classification_report(svc.predict(X_train), y_train))

0.7969924812030075
0.8032786885245902
0.9228215767634855


In [8]:
# save the models
import pickle as pkl
with open("rf_model.pkl", "wb") as f:
    pkl.dump(rf, f)
with open("svc_model.pkl", "wb") as f:
    pkl.dump(svc, f)